In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import QPSO
from QPSO import QPSO

In [11]:
data=load_iris()
n_inputs = 4
n_hidden = 50
n_classes = 3
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
X=data['data']
y=data['target']
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2)
Xtrain.shape
Xtest.shape


(30, 4)

In [46]:
df = pd.read_csv('cleanedData.csv')
featureList = list(df.columns)
featureList.remove('P. Habitable')
features = df[featureList].as_matrix()
target = df['P. Habitable'].as_matrix()
X = features
y = np.array(target, dtype=int)
n_inputs = 50
n_hidden = 50
n_classes = 2
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
X.shape

/home/azhar/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/home/azhar/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


(3875, 50)

In [12]:
# Forward propagation
def forward_prop(params):
    params=np.array(params)
    #print(type(params))
    W1 = params[0:n_inputs*n_hidden].reshape((n_inputs,n_hidden))
    b1 = params[n_inputs*n_hidden:n_inputs*n_hidden+n_hidden].reshape((n_hidden,))
    W2 = params[n_inputs*n_hidden+n_hidden:n_inputs*n_hidden+n_hidden+n_hidden*n_classes].reshape((n_hidden,n_classes))
    b2 = params[n_inputs*n_hidden+n_hidden+n_hidden*n_classes:n_inputs*n_hidden+n_hidden+n_hidden*n_classes + n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = Xtrain.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = 120 # Number of samples
    corect_logprobs = -np.log(probs[range(N), ytrain])
    loss = np.sum(corect_logprobs) / N

    return loss


In [75]:
#QPSO(self, costFunc, x0,bounds, num_particles, maxiter, verbose=False,num_dimensions):
#np.random.seed(6)
initial=np.random.rand(dimensions)
type(initial)
#bounds=[(-20,20),(-20,20)]
pos=QPSO(forward_prop,initial,num_particles=10000,maxiter=30,verbose=True)

iter: 0, best solution: -1
iter: 1, best solution: 1.8258258171701778
iter: 2, best solution: 1.8226770432767707
iter: 3, best solution: 1.8094340483268445
iter: 4, best solution: 1.8026180292791758
iter: 5, best solution: 1.5252113563318133
iter: 6, best solution: 1.471944159136156
iter: 7, best solution: 1.3642855384605208
iter: 8, best solution: 1.303643453194078


/home/azhar/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/home/azhar/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/home/azhar/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log


iter: 9, best solution: 1.1591294014899305
iter: 10, best solution: 1.107612963561677
iter: 11, best solution: 1.0525988580120962
iter: 12, best solution: 0.9829617911465862
iter: 13, best solution: 0.9062272296731251
iter: 14, best solution: 0.8655562180905821
iter: 15, best solution: 0.7299035907219036
iter: 16, best solution: 0.47791587165406185
iter: 17, best solution: 0.3431140764758961
iter: 18, best solution: 0.330951691109358
iter: 19, best solution: 0.330951691109358
iter: 20, best solution: 0.3286886478473058
iter: 21, best solution: 0.2716306411060756
iter: 22, best solution: 0.1369887193437504
iter: 23, best solution: 0.1369887193437504
iter: 24, best solution: 0.1369887193437504
iter: 25, best solution: 0.1369887193437504
iter: 26, best solution: 0.1369887193437504
iter: 27, best solution: 0.1369887193437504
iter: 28, best solution: 0.1369887193437504
iter: 29, best solution: 0.1369887193437504

FINAL SOLUTION:
   > [0.577724177131109, 0.5202667677791833, 0.594685394218183

In [14]:
len(pos.pos_best_g)


403

In [9]:
def predict(X, pos):
    pos=np.array(pos)
    W1 = pos[0:n_inputs*n_hidden].reshape((n_inputs,n_hidden))
    b1 = pos[n_inputs*n_hidden:n_inputs*n_hidden+n_hidden].reshape((n_hidden,))
    W2 = pos[n_inputs*n_hidden+n_hidden:n_inputs*n_hidden+n_hidden+n_hidden*n_classes].reshape((n_hidden,n_classes))
    b2 = pos[n_inputs*n_hidden+n_hidden+n_hidden*n_classes:n_inputs*n_hidden+n_hidden+n_hidden*n_classes + n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [76]:
(predict(Xtest, pos.pos_best_g) == ytest).mean()

0.9333333333333333

In [77]:
(predict(Xtrain, pos.pos_best_g) == ytrain).mean()

0.9583333333333334